# Route Optimization with Suitability Modelling <br> (Lab 3 - Part 1)

### GIS 5571: ArcGIS I <br> University of Minnesota

##### Luke Zaruba <br> November 30, 2022

<u>**NOTE:**</u> The workflow in this notebook is essentially the same exact workflow as the one I created in lab 2 (part 2), since I already added the functionality to iteratively test different weights in my model. For the purposes of this lab, I removed much of the preprocessing that went into preparing the data and also only created three different cost surfaces, rather than 10 as I did previously.

The more exhaustive workflow that was previously created can be found in the lab's repo [here](https://github.com/lukezaruba/GIS5571/blob/main/Lab2/Part%202/Lab2_Part2/Zaruba%20Lab2_2.ipynb). 

In [1]:
import arcpy

In [2]:
from arcpy import env
env.scratchWorkspace = env.scratchFolder
env.workspace = env.scratchWorkspace

## Reclassify Data and Run Model (Processed and Cleaned in Lab 2)

In [3]:
# Reclassify Landcover
lc_reclass = arcpy.sa.Reclassify("landcover_aoi", "Value", "1 100 5;101 2;102 2;103 2;104 3;105 2;106 3;107 3;108 4;109 1;110 1", "DATA")

In [4]:
# Reclassify Slope
slope_reclass = arcpy.sa.Reclassify("slope", "VALUE", "0 2.084853 5;2.084853 3.822231 4;3.822231 6.602035 3;6.602035 18.416204 2;18.416204 88.606262 1", "DATA") 

In [5]:
# Create Three Cost Surfaces to Test Different Model Weights
for i in [0.1, 0.5, 0.9]:
    # Set Weights
    slope_w_rnd = round(i, 1)
    landcover_w_rnd = round((1 - i), 1)
    
    # Calculate Cost and Save as New Raster
    cost = ((((arcpy.Raster(slope_reclass) * slope_w_rnd) + (arcpy.Raster(lc_reclass) * landcover_w_rnd)) * -1) + 6)

    # Create Cost Path
    cPath_name = fr"cPath_{str(slope_w_rnd)[2:3]}s_{str(landcover_w_rnd)[2:3]}lc"
    cPath = arcpy.sa.OptimalRegionConnections("origin_destination", cPath_name, in_cost_raster = cost)